# 0. install

In [ ]:
!pip install -q xgboost
!pip install -q -U keras-tuner
!pip install -q pandas tensorflow scikit-learn seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


# 1. Import libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import keras_tuner as kt

#2. DATA

In [ ]:
qt = pd.read_csv('/content/drive/MyDrive/TBTL/qt-public.csv')
th = pd.read_csv('/content/drive/MyDrive/TBTL/th-public.csv')
ck = pd.read_csv('/content/drive/MyDrive/TBTL/ck-public.csv')

qt = qt.rename(columns={"hash": "student_id", "diemqt": "qt"})
th = th.rename(columns={"hash": "student_id", "diemth": "TH"})
ck = ck.rename(columns={"hash": "student_id", "diemck": "CK"})

# Merge
merged = qt.merge(th, on='student_id', how='outer') \
           .merge(ck, on='student_id', how='outer')

# Convert to numeric
for col in ['qt', 'TH', 'CK']:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

# Tính TBTL
merged['tbtl'] = merged['CK'] * 0.5 + merged['TH'] * 0.3 + merged['qt'] * 0.2
tbtl = merged[['student_id', 'tbtl']].dropna()

tbtl

,student_id,tbtl
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.75
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7.20
2,01122b3ef7e59b84189e65985305f575d6bdf83c,6.15
3,013de369c439ab0ead8aa7da64423aa395a8be39,7.50
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,5.60
...,...,...
756,840d07858c03f80f4695056e2cc7d0c474b83a25,6.35
757,844f5db2e7e31ae51eba025480679ed7e4708ac6,7.15
758,845acd04a77b3d1b623f255d9f9f8eae90892dab,5.95
759,8460eaaf887a6289fb156f7562fb739ba8e9629e,8.50


# 3. Xử lý dữ liệu submissions


In [ ]:
submissions = pd.read_csv('/content/drive/MyDrive/TBTL/annonimized.csv')
submissions = submissions.rename(columns={
    "concat('it001',`assignment_id`)": "assignment_id",
    "concat('it001',`problem_id`)": "problem_id",
    "concat('it001', username)": "student_id",
    "concat('it001',`language_id`)": "language_id"
})

# Chuyển đổi kiểu dữ liệu
submissions['is_final'] = submissions['is_final'].apply(lambda x: x == 1)
submissions['pre_score'] = submissions['pre_score'].astype(float)
submissions['created_at'] = pd.to_datetime(submissions['created_at'], format='%m-%d %H:%M:%S', errors='coerce')
submissions['updated_at'] = pd.to_datetime(submissions['updated_at'], format='%m-%d %H:%M:%S', errors='coerce')

# Trích xuất wrong_count
def extract_wrong_count(text):
    if pd.isna(text):
        return 0
    match = re.search(r'wrong_count:\s*(\d+)', str(text))
    return int(match.group(1)) if match else 0

submissions['wrong_count'] = submissions['judgement'].apply(extract_wrong_count)
submissions['pre_score_percentage'] = submissions['pre_score'] / 100


print(f"✅ Submissions data processed: {len(submissions)} records")

✅ Submissions data processed: 295198 records


In [ ]:
submissions

,assignment_id,problem_id,student_id,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,judgement,wrong_count,pre_score_percentage
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,False,SCORE,0.0,100,it0012,1900-10-09 08:02:04,1900-10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0,0.0
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,False,SCORE,0.0,100,it0012,1900-10-09 08:04:41,1900-10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0,0.0
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,True,SCORE,10000.0,100,it0012,1900-10-09 08:06:49,1900-10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0,100.0
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,True,SCORE,10000.0,100,it0012,1900-10-09 08:47:52,1900-10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0,100.0
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,True,SCORE,10000.0,100,it0012,1900-10-09 09:19:35,1900-10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,True,SCORE,10000.0,100,it0012,1900-01-15 16:03:43,1900-01-15 16:03:53,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",0,100.0
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,False,Compilation Error,0.0,100,it0012,1900-01-15 16:04:07,1900-01-15 16:05:08,"{""times"":[],""mems"":[],""verdicts"":{""\n\nIn func...",0,0.0
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,True,SCORE,10000.0,100,it0012,1900-01-15 16:04:58,1900-01-15 16:05:08,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",0,100.0
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,True,SCORE,10000.0,100,it0012,1900-01-15 16:05:13,1900-01-15 16:05:22,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":[0...",0,100.0


# 4. Tổng hợp về mức sinh viên (Features)

In [ ]:
print("\n Advanced feature engineering...")

def create_features_v1_basic(df):
    """Basic feature set"""
    features = df.groupby('student_id').agg(
        total_sub=('assignment_id', 'count'),
        avg_pass=('pre_score_percentage', 'mean'),
        pass_ratio=('is_final', 'mean'),
    ).reset_index()

    features['pass_quality'] = features['avg_pass'] * features['pass_ratio']
    features['submission_efficiency'] = features['avg_pass'] / (features['total_sub'] + 1)
    features['submission_strength'] = features['avg_pass'] * features['total_sub']

    return features

def create_features_v2_enhanced(df):
    """Enhanced feature set with log transforms and noise reduction"""
    # Basic aggregations
    features = df.groupby('student_id').agg(
        total_sub=('assignment_id', 'count'),
        unique_assignments=('assignment_id', 'nunique'),
        unique_problems=('problem_id', 'nunique'),

        # Score statistics
        avg_pass=('pre_score_percentage', 'mean'),
        median_pass=('pre_score_percentage', 'median'),
        std_pass=('pre_score_percentage', 'std'),
        max_pass=('pre_score_percentage', 'max'),
        min_pass=('pre_score_percentage', 'min'),

        # Success metrics
        pass_ratio=('is_final', 'mean'),
        total_passed=('is_final', 'sum'),

        # Error analysis
        avg_wrong=('wrong_count', 'mean'),
        total_wrong=('wrong_count', 'sum'),
        max_wrong=('wrong_count', 'max'),
    ).reset_index()

    # Fill NaN values
    features = features.fillna(0)

    # Log transforms for skewed distributions
    features['log_total_sub'] = np.log1p(features['total_sub'])
    features['log_unique_assignments'] = np.log1p(features['unique_assignments'])
    features['log_unique_problems'] = np.log1p(features['unique_problems'])
    features['log_total_passed'] = np.log1p(features['total_passed'])

    # Sqrt transforms
    features['sqrt_total_sub'] = np.sqrt(features['total_sub'])
    features['sqrt_submission_strength'] = np.sqrt(features['avg_pass'] * features['total_sub'])

    # Enhanced derived features
    features['pass_quality'] = features['avg_pass'] * features['pass_ratio']
    features['submission_efficiency'] = features['avg_pass'] / (features['total_sub'] + 1)
    features['submission_strength'] = features['avg_pass'] * features['total_sub']
    features['consistency'] = 1 / (features['std_pass'] + 0.1)  # Higher = more consistent
    features['problem_diversity'] = features['unique_problems'] / (features['unique_assignments'] + 1)
    features['error_rate'] = features['avg_wrong'] / (features['avg_pass'] + 0.1)
    features['improvement_potential'] = features['max_pass'] - features['min_pass']

    # Activity patterns
    features['submissions_per_assignment'] = features['total_sub'] / (features['unique_assignments'] + 1)
    features['success_efficiency'] = features['total_passed'] / (features['total_sub'] + 1)

    return features

def create_features_v3_robust(df):
    """Robust feature set with outlier handling"""
    features = create_features_v2_enhanced(df)

    # Outlier clipping (robust approach)
    numeric_cols = features.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col != 'student_id']

    for col in numeric_cols:
        if col in features.columns:
            Q1 = features[col].quantile(0.25)
            Q3 = features[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            features[col] = np.clip(features[col], lower_bound, upper_bound)

    return features

# Create different feature sets
features_v1 = create_features_v1_basic(submissions)
features_v2 = create_features_v2_enhanced(submissions)
features_v3 = create_features_v3_robust(submissions)

print(f"✅ Feature sets created:")
print(f"   V1 (Basic): {features_v1.shape[1]-1} features")
print(f"   V2 (Enhanced): {features_v2.shape[1]-1} features")
print(f"   V3 (Robust): {features_v3.shape[1]-1} features")


 Advanced feature engineering...
✅ Feature sets created:
   V1 (Basic): 6 features
   V2 (Enhanced): 28 features
   V3 (Robust): 28 features


In [ ]:
features_v1

,student_id,total_sub,avg_pass,pass_ratio,pass_quality,submission_efficiency,submission_strength
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,147,55.041497,0.312925,17.223870,0.371902,8091.10
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,259,54.885521,0.301158,16.529230,0.211098,14215.35
2,01122b3ef7e59b84189e65985305f575d6bdf83c,195,59.737538,0.343590,20.525206,0.304783,11648.82
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,100,59.527600,0.470000,27.977972,0.589382,5952.76
4,013de369c439ab0ead8aa7da64423aa395a8be39,107,64.744486,0.616822,39.935851,0.599486,6927.66
...,...,...,...,...,...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,74,55.810811,0.459459,25.642805,0.744144,4130.00
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,260,62.045577,0.300000,18.613673,0.237723,16131.85
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,437,49.984668,0.235698,11.781283,0.114120,21843.30
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,94,43.433298,0.361702,15.709916,0.457193,4082.73


In [ ]:
features_v2

,student_id,total_sub,unique_assignments,unique_problems,avg_pass,median_pass,std_pass,max_pass,min_pass,pass_ratio,...,sqrt_submission_strength,pass_quality,submission_efficiency,submission_strength,consistency,problem_diversity,error_rate,improvement_potential,submissions_per_assignment,success_efficiency
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,147,7,46,55.041497,76.92,45.414358,100.0,0.0,0.312925,...,89.950542,17.223870,0.371902,8091.10,0.021971,5.750000,0.0,100.0,18.375000,0.310811
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,259,9,78,54.885521,70.00,43.237609,100.0,0.0,0.301158,...,119.228143,16.529230,0.211098,14215.35,0.023075,7.800000,0.0,100.0,25.900000,0.300000
2,01122b3ef7e59b84189e65985305f575d6bdf83c,195,7,66,59.737538,80.95,41.699366,100.0,0.0,0.343590,...,107.929699,20.525206,0.304783,11648.82,0.023924,8.250000,0.0,100.0,24.375000,0.341837
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,100,4,47,59.527600,100.00,46.403021,100.0,0.0,0.470000,...,77.154131,27.977972,0.589382,5952.76,0.021504,9.400000,0.0,100.0,20.000000,0.465347
4,013de369c439ab0ead8aa7da64423aa395a8be39,107,8,52,64.744486,100.00,44.096659,100.0,0.0,0.616822,...,83.232566,39.935851,0.599486,6927.66,0.022626,5.777778,0.0,100.0,11.888889,0.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,74,9,34,55.810811,100.00,49.435353,100.0,0.0,0.459459,...,64.265076,25.642805,0.744144,4130.00,0.020188,3.400000,0.0,100.0,7.400000,0.453333
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,260,6,77,62.045577,80.00,39.070873,100.0,0.0,0.300000,...,127.011220,18.613673,0.237723,16131.85,0.025529,11.000000,0.0,100.0,37.142857,0.298851
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,437,6,100,49.984668,55.55,41.870943,100.0,0.0,0.235698,...,147.794790,11.781283,0.114120,21843.30,0.023826,14.285714,0.0,100.0,62.428571,0.235160
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,94,7,34,43.433298,27.77,44.433280,100.0,0.0,0.361702,...,63.896244,15.709916,0.457193,4082.73,0.022455,4.250000,0.0,100.0,11.750000,0.357895


In [ ]:
features_v3

,student_id,total_sub,unique_assignments,unique_problems,avg_pass,median_pass,std_pass,max_pass,min_pass,pass_ratio,...,sqrt_submission_strength,pass_quality,submission_efficiency,submission_strength,consistency,problem_diversity,error_rate,improvement_potential,submissions_per_assignment,success_efficiency
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,147,7.0,46,55.041497,76.92,45.414358,100.0,0.0,0.312925,...,89.950542,17.223870,0.371902,8091.10,0.021971,5.750000,0.0,100.0,18.375000,0.310811
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,259,9.0,78,54.885521,70.00,43.237609,100.0,0.0,0.301158,...,119.228143,16.529230,0.211098,14215.35,0.023075,7.800000,0.0,100.0,25.900000,0.300000
2,01122b3ef7e59b84189e65985305f575d6bdf83c,195,7.0,66,59.737538,80.95,41.699366,100.0,0.0,0.343590,...,107.929699,20.525206,0.304783,11648.82,0.023924,8.250000,0.0,100.0,24.375000,0.341837
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,100,4.0,47,59.527600,100.00,46.403021,100.0,0.0,0.470000,...,77.154131,27.977972,0.589382,5952.76,0.021504,9.400000,0.0,100.0,20.000000,0.465347
4,013de369c439ab0ead8aa7da64423aa395a8be39,107,8.0,52,64.744486,100.00,44.096659,100.0,0.0,0.616822,...,83.232566,39.935851,0.599486,6927.66,0.022626,5.777778,0.0,100.0,11.888889,0.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,74,9.0,34,55.810811,100.00,49.435353,100.0,0.0,0.459459,...,64.265076,25.642805,0.744144,4130.00,0.020188,3.400000,0.0,100.0,7.400000,0.453333
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,260,6.0,77,62.045577,80.00,39.070873,100.0,0.0,0.300000,...,127.011220,18.613673,0.237723,16131.85,0.025529,11.000000,0.0,100.0,37.142857,0.298851
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,437,6.0,100,49.984668,55.55,41.870943,100.0,0.0,0.235698,...,147.794790,11.781283,0.114120,21843.30,0.023826,14.285714,0.0,100.0,56.666667,0.235160
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,94,7.0,34,43.433298,27.77,44.433280,100.0,0.0,0.361702,...,63.896244,15.709916,0.457193,4082.73,0.022455,4.250000,0.0,100.0,11.750000,0.357895


# 5. Đánh giá model với K-fold cross-validation

In [ ]:
def evaluate_model_kfold(X, y, model, cv=5, scaler_type='standard'):
    """Evaluate model using k-fold cross-validation"""
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)

    r2_scores = []
    mae_scores = []
    rmse_scores = []

    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        # Scale features
        if scaler_type == 'standard':
            scaler = StandardScaler()
        elif scaler_type == 'robust':
            scaler = RobustScaler()

        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled = scaler.transform(X_val_fold)

        # Train and predict
        model_clone = model.__class__(**model.get_params())
        model_clone.fit(X_train_scaled, y_train_fold)
        y_pred = model_clone.predict(X_val_scaled)

        # Calculate metrics
        r2_scores.append(r2_score(y_val_fold, y_pred))
        mae_scores.append(mean_absolute_error(y_val_fold, y_pred))
        rmse_scores.append(np.sqrt(mean_squared_error(y_val_fold, y_pred)))

    return {
        'r2_mean': np.mean(r2_scores),
        'r2_std': np.std(r2_scores),
        'mae_mean': np.mean(mae_scores),
        'mae_std': np.std(mae_scores),
        'rmse_mean': np.mean(rmse_scores),
        'rmse_std': np.std(rmse_scores)
    }

# 6. Xây dựng hàm so sánh các Model

In [ ]:
def compare_feature_sets():
    """Compare different feature sets and models"""

    feature_sets = {
        'Basic (V1)': features_v1,
        'Enhanced (V2)': features_v2,
        'Robust (V3)': features_v3
    }

    models = {
        'XGBoost': xgb.XGBRegressor(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        ),
        'RandomForest': RandomForestRegressor(
            n_estimators=200,
            max_depth=10,
            random_state=42
        ),
        'Ridge': Ridge(alpha=1.0),
        'Lasso': Lasso(alpha=0.1)
    }

    results = []

    for feature_name, features in feature_sets.items():
        # Merge with target
        data = features.merge(tbtl, on='student_id', how='inner').fillna(0)

        if len(data) == 0:
            continue

        X = data.drop(columns=['student_id', 'tbtl']).values
        y = data['tbtl'].values

        print(f"\n📊 Evaluating {feature_name} ({X.shape[1]} features, {len(data)} samples)")

        for model_name, model in models.items():
            try:
                scaler_type = 'robust' if feature_name == 'Robust (V3)' else 'standard'
                metrics = evaluate_model_kfold(X, y, model, cv=5, scaler_type=scaler_type)

                results.append({
                    'Feature Set': feature_name,
                    'Model': model_name,
                    'R² (mean)': metrics['r2_mean'],
                    'R² (std)': metrics['r2_std'],
                    'MAE (mean)': metrics['mae_mean'],
                    'MAE (std)': metrics['mae_std'],
                    'RMSE (mean)': metrics['rmse_mean'],
                    'RMSE (std)': metrics['rmse_std']
                })

                print(f"   {model_name:12} | R²: {metrics['r2_mean']:.4f}±{metrics['r2_std']:.4f} | "
                      f"MAE: {metrics['mae_mean']:.3f}±{metrics['mae_std']:.3f}")

            except Exception as e:
                print(f"   {model_name:12} | Error: {str(e)}")

    return pd.DataFrame(results)

# 7. Bắt đầu so sánh:

In [ ]:
print("\n🚀 Bắt đầu so sánh với K-fold validation...")
results_df = compare_feature_sets()

# Display results
print("\n📈 FINAL RESULTS SUMMARY:")
print("="*80)
results_sorted = results_df.sort_values('R² (mean)', ascending=False)
print(results_sorted.to_string(index=False, float_format='%.4f'))



🚀 Bắt đầu so sánh với K-fold validation...

📊 Evaluating Basic (V1) (6 features, 753 samples)
   XGBoost      | R²: 0.0032±0.0979 | MAE: 1.326±0.048
   RandomForest | R²: 0.0983±0.0709 | MAE: 1.263±0.052
   Ridge        | R²: 0.0763±0.1202 | MAE: 1.271±0.076
   Lasso        | R²: 0.0559±0.0522 | MAE: 1.321±0.064

📊 Evaluating Enhanced (V2) (28 features, 753 samples)
   XGBoost      | R²: 0.1777±0.0776 | MAE: 1.181±0.046
   RandomForest | R²: 0.2706±0.0727 | MAE: 1.128±0.058
   Ridge        | R²: 0.1545±0.1296 | MAE: 1.225±0.086
   Lasso        | R²: 0.0939±0.0579 | MAE: 1.283±0.067

📊 Evaluating Robust (V3) (28 features, 753 samples)
   XGBoost      | R²: 0.1909±0.0814 | MAE: 1.172±0.042
   RandomForest | R²: 0.2661±0.0672 | MAE: 1.135±0.058
   Ridge        | R²: 0.1327±0.0990 | MAE: 1.234±0.091
   Lasso        | R²: 0.1086±0.0522 | MAE: 1.277±0.064

📈 FINAL RESULTS SUMMARY:
  Feature Set        Model  R² (mean)  R² (std)  MAE (mean)  MAE (std)  RMSE (mean)  RMSE (std)
Enhanced (V2) R

# 8. Chuẩn bị training data: ( lưu ý: cần chạy bài nào thì chỉ chạy data cho đúng 1 dòng đó thôi !!!)

a) TBTL:

In [ ]:
print("\n Đang gộp đặc trưng với mục tiêu...")
training_data = features_v2.merge(tbtl, on='student_id', how='inner').fillna(0)

X = training_data.drop(columns=['student_id', 'tbtl'])
y = training_data['tbtl']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


 Đang gộp đặc trưng với mục tiêu...


b) Quá trình:

In [ ]:
print("\n Đang gộp đặc trưng với mục tiêu...")
training_data = features_v2.merge(qt, on='student_id', how='inner').fillna(0)

X = training_data.drop(columns=['student_id', 'qt'])
y = training_data['tbtl']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

c) Thực hành:

In [ ]:
print("\n Đang gộp đặc trưng với mục tiêu...")
training_data = features_v2.merge(th, on='student_id', how='inner').fillna(0)

X = training_data.drop(columns=['student_id', 'TH'])
y = training_data['tbtl']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

d) Cuối kỳ:

In [ ]:
print("\n Đang gộp đặc trưng với mục tiêu...")
training_data = features_v2.merge(ck, on='student_id', how='inner').fillna(0)

X = training_data.drop(columns=['student_id', 'CK'])
y = training_data['tbtl']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 9. Train model sau khi so sánh( ở đây chọn RandomForest do nó có R2 nhỏ nhất ):

In [ ]:
print("\n🌲 Đang huấn luyện mô hình RandomForest...")
final_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

final_model.fit(X_train_scaled, y_train)


🌲 Đang huấn luyện mô hình RandomForest...


RandomForestRegressor(max_depth=12, max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=5, n_estimators=500, n_jobs=-1,
                      random_state=42)

# 10. Đánh giá mô hình:

In [ ]:
print("\n📊 Đang đánh giá mô hình...")
y_val_pred = final_model.predict(X_val_scaled)
r2_val = r2_score(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"✅ Kết quả đánh giá:")
print(f"   R² Score:  {r2_val:.4f}")
print(f"   MAE:       {mae_val:.4f}")


📊 Đang đánh giá mô hình...
✅ Kết quả đánh giá:
   R² Score:  0.3332
   MAE:       1.0518


# 11. Bắt đầu xuất kết quả dự đoán:

In [ ]:
print("\n💾 Đang tạo dự đoán cuối cùng...")
all_students_features = features_v2.copy()
X_all = all_students_features.drop(columns=['student_id'])
X_all_scaled = scaler.transform(X_all)
predictions = final_model.predict(X_all_scaled)

# Tạo dataframe nộp bài
submission = pd.DataFrame({
    'student_id': all_students_features['student_id'],
    'tbtl': predictions
})

# Làm tròn số
submission['tbtl'] = submission['tbtl'].round(3)

# Lưu vào file CSV
output_file = '/content/drive/MyDrive/TBTL/results.csv'
submission.to_csv(output_file, index=False)
print(f"✅ Đã lưu vào: {output_file}")
print(f"📊 Tổng số dự đoán: {len(submission)}")


💾 Đang tạo dự đoán cuối cùng...
✅ Đã lưu vào: /content/drive/MyDrive/TBTL/tbtl_submission.csv
📊 Tổng số dự đoán: 1489
